### 패키지 설치

In [ ]:
!pip install ratsnlp -q

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

### 모델 환경설정

In [ ]:
import torch

from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args = ClassificationTrainArguments(
    pretrained_model_name= "beomi/kcbert-base",
    downstream_corpus_name="nsmc",
    downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-doccls",
    downstream_corpus_root_dir="/content/Korpora",
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate = 5e-5,
    max_seq_length = 128,
    epochs = 3,
    tpu_cores = 0 if torch.cuda.is_available() else 8,
    seed = 7
)

- pretrained_model_name : 프리트레인 마친 언어 모델의 이름(단, 해당 모델이 허깅페이스 모델 허브에 등록되어 있어야 함)

- downstream_corpus_name : 다운스트림 데이터의 이름

- dowstream_corpus_root_dir : 다운스트림 데이터를 내려받을 위치, 입력하지 않으면 코랩 환경 로컬에 저장

- batch_size : 하드웨어 가속기로 GPU를 선택했다면 32, TPU라면 4로 설정, 코랩 환경에서 TPU는 보통 8개의 코어가 할당되는데 batch_size는 코어별로 적용되는 배치 크기

- learning_rate : 러닝 레이트, 1회 스텝에서 모델을 얼마나 업데이트 할지에 관한 크기를 가리킴

- max_seq_length : 토큰 기준 입력 문장 최대 길이. 이보다 긴 문장은 자르고, 짧은 문장은 동일한 길이를 만들기 위해 스페셜 토큰 ([PAD])를 붙임

### 랜덤 시드 고정

In [ ]:
from ratsnlp import nlpbook

nlpbook.set_seed(args)

In [ ]:
nlpbook.set_logger(args) # 각종 로그 출력

### 말뭉치 다운로드

In [ ]:
from Korpora import Korpora

Korpora.fetch(
    corpus_name = args.downstream_corpus_name,
    root_dir = args.downstream_corpus_root_dir,
    force_download=True,
)

### 토크나이저 준비

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

### 데이터 전처리

In [65]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset

corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = "train"
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_BertTokenizer_128_nsmc_document-classification [took 11.658 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_BertTokenizer_128_nsmc_document-classification [took 11.658 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_BertTokenizer_128_nsmc_document-classification [took 11.658 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_BertTokenizer_128_nsmc_document-classification [took 11.658 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_train_BertTokenizer_128_nsmc_document-classification [took 11.658 s]


- ClassificationDataset은 corpus와 토크나이저를 포함

- NsmcCorpus는 csv 파일 형식의 nsmc 데이터를 문장과 레이블로 로드

- ClassificationDataset은 NsmcCorpus가 넘겨준 문장과 레이블 각각을 모델이 학습할 수 있는 형태로 가공

- input_ids, attention_mask, token_types_ids, label(정수로 바뀐 레이블 정보) 확인 가능

- ratsgo.github.io/nlpbook/docs/doc_cls/detail

In [66]:
train_dataset[0]

ClassificationFeatures(input_ids=[2, 2170, 832, 5045, 17, 17, 7992, 29734, 4040, 10720, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], attention_mask=[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], token_type_ids=[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### 학습 데이터 로더 구축

In [67]:
from torch.utils.data import DataLoader, RandomSampler

train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement = False),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers
)

- sampler : 샘플링 방식을 정의, 여기서 만든 데이터 로더는 배치를 만들 때 전체 인스턴스 가운데 batch_size 개수 만큼을 비복원 랜덤 추출

- collate_fn : 이렇게 뽑은 인스턴스들을 배치로 만드는 역할을 하는 함수. nlpbook.data_collator는 같은 배치에서 인스턴스가 여럿일 때 이를 input_ids, attention_mask 등 종류별로 모으고 파이토치가 요구하는 텐서(tensor) 형태로 바꾸는 역할을 수행

### 평가용 데이터 로더 구축

In [68]:
from torch.utils.data import SequentialSampler

val_dataset = ClassificationDataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = "test"
)

INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_BertTokenizer_128_nsmc_document-classification [took 4.170 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_BertTokenizer_128_nsmc_document-classification [took 4.170 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_BertTokenizer_128_nsmc_document-classification [took 4.170 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_BertTokenizer_128_nsmc_document-classification [took 4.170 s]
INFO:ratsnlp:Loading features from cached file /content/Korpora/nsmc/cached_test_BertTokenizer_128_nsmc_document-classification [took 4.170 s]


- SequentialSampler : 평가용 데이터 로더는 학습용 데이터 로더와 다른 샘플러 사용, 학습 때 배치 구성은 랜덤으로 하는 것이 좋지만, 평가할 때는 평가용 데이터 전체를 쓰기 때문에 굳이 랜덤으로 구성할 이유가 없음

In [69]:
val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers
)

### 모델 불러오기

In [70]:
from transformers import BertConfig, BertForSequenceClassification

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = corpus.num_labels
)

model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config = pretrained_model_config
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

### 모델 학습시키기

In [71]:
# Task 정의

from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [72]:
trainer = nlpbook.get_trainer(args)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True, used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [73]:
#trainer.fit(
    #task,
    #train_dataloaders = train_dataloader,
    #val_dataloaders = val_dataloader)

### 실전 투입

In [81]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args = ClassificationDeployArguments(
    pretrained_model_name = "beomi/kcbert-base",
    downstream_model_dir="/gdrive/MyDrive/nlpbook/checkpoint-doccls",
    max_seq_length = 128
)

downstream_model_checkpoint_fpath: /gdrive/MyDrive/nlpbook/checkpoint-doccls/epoch=1-val_loss=0.26.ckpt


In [83]:
from transformers import BertTokenizer

# 토크나이저 로드

tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False
)

In [79]:
import torch

# 체크포인트 로드

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device("cpu")
)

In [82]:
from transformers import BertConfig

# 파인튜닝 때 사용한 프리트레인 모델 설정값 로드(bert 설정 로드)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt["state_dict"]["model.classifier.bias"].shape.numel(),
)

In [84]:
from transformers import BertForSequenceClassification 

# 모델 초기화

model = BertForSequenceClassification(pretrained_model_config)

In [85]:
# 체크포인트 주입

model.load_state_dict({k.replace("model.",""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})

<All keys matched successfully>

In [ ]:
model.eval() # 평가 모드로 전환

### 모델 출력값 만들고 후처리

In [88]:
def inference_fn(sentence):
  inputs = tokenizer(
      [sentence],
      max_length = args.max_seq_length,
      padding = "max_length",
      truncation = True
  )

  with torch.no_grad():
    outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()}) # input을 파이토치 텐서로 변경
    prob = outputs.logits.softmax(dim=1)                               # 로짓에 softmax
    positive_prob = round(prob[0][1].item(),4)                
    negative_prob = round(prob[0][0].item(),4)
    pred = "긍정 (positive) " if torch.argmax(prob) == 1 else "부정 (negative) " # 예측 확률의 최댓값 위치에 따라 pred 제작

  return{
      'sentence' : sentence,
      'pred' : pred,
      'positive_data' : f"긍정 {positive_prob}",
      'negative_data' : f"부정 {negative_prob}",
      'positive_data' : f"{positive_prob * 100}%",
      'negative_data' : f"{negative_prob * 100}%",
  }

- 로짓 : 로짓 함수는 0에서 1까지의 확률값과 -∞에서 ∞ 사이의 확률값을 표현해주는 함수로, X축이 아니라 Y축에서 0과 1 사이의 값을 제한하는 시그모이드 함수에 대한 역함수 

- 로짓 함수는 0 - 1의 도메인 내에 존재하기 때문에 이 함수는 확률을 이해하는 데 가장 일반적으로 사용

### 웹 서비스 시작하기

In [93]:
from ratsnlp.nlpbook.classification import get_web_service_app

app = get_web_service_app(inference_fn)
app.run() # 실행 안됨

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://7073-34-121-220-248.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:10:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:10:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:11:09] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:11:09] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:12:21] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:12:22] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:13:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:13:35] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:13:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:13:40] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:15:47] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Nov/2022 14:15:47] "GET /favicon.ico HTTP/1.1" 404 -
